##**Bibliotecas necessárias**##

In [1]:
# Importando bibliotecas necessárias
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error

# COLETA E EXPLORAÇÃO DOS DADOS

In [2]:
# Carregando o dataset
df = pd.read_csv('autos.csv', encoding='latin1')

In [3]:
# Excluindo colunas irrelevantes
df.drop(['name', 'nrOfPictures', 'postalCode', 'dateCreated', 'lastSeen', 'dateCrawled'], axis=1, inplace=True)

In [4]:
# Tratamento de dados nulos
df['fuelType'].fillna('Desconhecido', inplace=True)
df['gearbox'].fillna('Desconhecido', inplace=True)
df['notRepairedDamage'].fillna('Desconhecido', inplace=True)
df['powerPS'].fillna(df['powerPS'].median(), inplace=True)
df['kilometer'].fillna(df['kilometer'].median(), inplace=True)

In [5]:
# Remover outliers no preço
df = df[df['price'] < 150000]  # Ajuste conforme necessário

In [6]:
# Agrupando marcas com menos de 500 ocorrências em 'Outros'
brand_counts = df['brand'].value_counts()
brands_to_replace = list(brand_counts[brand_counts < 500].index)
df['brand'] = df['brand'].replace(brands_to_replace, 'Outros')

<ipython-input-6-34d389ea1ebd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['brand'] = df['brand'].replace(brands_to_replace, 'Outros')


In [7]:
# Separando variáveis independentes (X) e dependente (y) usando todo o conjunto de dados
df_sample = df.sample(frac=0.8, random_state=42)
X_sample = df_sample.drop('price', axis=1)
y_sample = df_sample['price']


In [8]:
# Convertendo variáveis categóricas em dummies (one-hot encoding)
X_sample = pd.get_dummies(X_sample, drop_first=True)

In [9]:
# Dividindo o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

In [10]:
# Padronizando as variáveis numéricas após a divisão
numeric_cols = ['yearOfRegistration', 'powerPS', 'kilometer', 'monthOfRegistration']
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

#Ajuste do Modelo Random Forest com Grid Search e K-Fold

In [11]:
# Definindo o modelo Random Forest
rf = RandomForestRegressor(random_state=42)

In [12]:
# Definindo o espaço de hiperparâmetros para GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

In [13]:
# Definindo o K-fold com 5 divisões
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [14]:
# Definindo o scoring como MAE (Mean Absolute Error)
scorer = 'neg_mean_absolute_error'


In [15]:
# Criando o GridSearchCV com o modelo Random Forest e o K-fold
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=kfold,
                           scoring=scorer, n_jobs=-1)

#Avaliação do Modelo no Conjunto de Dados de Teste

In [16]:
# Ajustando o GridSearchCV aos dados de treinamento
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
80 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", l

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [20, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='neg_mean_absolute_error')

In [17]:
# Extraindo os melhores parâmetros e o melhor resultado
best_model = grid_search.best_estimator_
print("Melhores parâmetros:", grid_search.best_params_)

Melhores parâmetros: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [20]:
# Fazendo previsões e calculando o MAE nos dados de teste
y_pred = best_model.predict(X_test)
# Criando um DataFrame com os preços reais e previstos
predictions_df = pd.DataFrame({'Preço Real': y_test, 'Preço Previsto': y_pred})
# Exibindo as primeiras linhas do DataFrame de previsões
print(predictions_df.head())

mae = mean_absolute_error(y_test, y_pred)
print("MAE nos dados de teste:", mae)
print(f"Erro Absoluto Médio (MAE) nos dados de teste: {mae:.2f}")

        Preço Real  Preço Previsto
96467         8000     8649.328333
66909         1650     2509.500000
162483        2799     4211.627105
57714         3950     4761.665000
117444        3500     3714.782500
MAE nos dados de teste: 1301.65144117893
Erro Absoluto Médio (MAE) nos dados de teste: 1301.65
